In [1]:
import os
import pyspark
conf = pyspark.SparkConf()

conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','8g')
sc = pyspark.SparkContext(conf=conf)

spark = pyspark.SQLContext.getOrCreate(sc)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/14 16:55:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/envs/bigdata/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
from pyspark.sql.functions import *
from haversine import haversine, Unit
restaurants = spark.read.option("inferSchema", "true").option("header", "true").option("delimiter",";").csv("shared/data/hw2_data/Restaurants_in_Durham_County_NC.csv")
restaurants = restaurants.filter((restaurants.Status=="ACTIVE") & (restaurants.Rpt_Area_Desc=="Food Service"))
foreclosure = spark.read.option("header","true").option("inferSchema","true").json("shared/data/hw2_data/durham-nc-foreclosure-2006-2016.json")
restaurants1 = restaurants.withColumn("geolocation_converted",split(col("geolocation"),",").cast("array<double>")).select("ID","geolocation_converted")
foreclosure1 = foreclosure.select("recordid","fields.geocode")
def distance_calc(loc1, loc2):
    return float(haversine(loc1, loc2, unit=Unit.DEGREES))
dist_udf = udf(distance_calc)
join = restaurants1.dropna().crossJoin(foreclosure1.dropna())
join_with_distance = join.withColumn("distance",dist_udf("geolocation_converted","geocode").cast("Double"))
total_fc_by_res = join_with_distance.filter(col("distance")<0.0167).groupBy("ID").count()
final_df = restaurants.alias("r").join(total_fc_by_res.alias("t"),col("r.ID") == col("t.ID"),"inner")
final_df.select("r.ID","Premise_name","count")

22/04/14 16:57:53 WARN ExtractPythonUDFFromJoinCondition: The join condition:(cast(distance_calc(geolocation_converted#141, geocode#168) as double) < 0.0167) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.
22/04/14 16:58:16 WARN ExtractPythonUDFFromJoinCondition: The join condition:(cast(distance_calc(geolocation_converted#141, geocode#168) as double) < 0.0167) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


ID,Premise_name,count
147244,LITTLE CAESARS #2,110
58358,TARGET MEAT MARKET,8
57180,TOAST,278
56081,INTERNATIONAL DEL...,43
57655,KROGER R 332 DELI,15
155004,OVAL PARK GRILLE,38
58318,TOM INSCOE WHOLES...,5
99093,TATER BREAD CAFE,604
57048,KFC,51
180224,ASHANTI CATERING ...,2
